Sensitivity analysis
============

__Goal__:
 - run sensitivity analysis to show the impact of a given parameter on the SMRT output
  
__Learning__: 
 

SMRT is able to iterate on several arguments when it is unambiguous. For instance, a sensor with multiple frequencies, angles or polarizations is automatically understood. The `result` contains all the values which can be accessed with arguments in TbV() and similar functions. E.g. TbV(frequency=37e9)

This is similar when a list of snowpacks is given to `run`. The `result` contains all the computations. The 'snowpack' dimension is automatically added but we can also propose a custom name for this dimension.

In the recent version,a pandas DataFrame with a snowpack column can be given to `run`. The result once converted to a dataframe contains all the column of the original DataFrame. This is the most advanced and powerful way to conduct sensitivity analysis.

In the following, we show different approaches to conduct sensitivity studies that  you can run and then apply to a study case of your choice:
 - take the Dome C snowpack and study the sensitivity of TbH 55° to superficial density
 - take any snowpack previously defined and investigated the sensivitiy to liquid_water
 - etc
 

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib notebook

from smrt import make_model, make_snowpack, sensor_list

Build a list of snowpack
--------------------------------

The key idea is to build a list of snowpack or a DataFrame. E.g. we want to test the sensitivity of TB's to the radius. We first build a list of snowpack with different radius.

In [ ]:
# prepare the snowpack
density = 300.0
radius = np.arange(0.05, 0.5, 0.01) *1e-3  # from 0 to 0.5mm

# the NAIVE APPROACH:

snowpack = list()
for x in radius:
    sp = make_snowpack([1000.0], "sticky_hard_spheres", 
                       density=density, temperature=265, 
                       radius=x, stickiness=0.15)
    snowpack.append(sp)

In simple cases (as this one), it is easier to use "list comprehension", a nice python feature to create list.

In [ ]:
# a BETTER APPROACH with list comprehension
snowpack = [make_snowpack([1000.0], "sticky_hard_spheres", 
                          density=density, temperature=265,
                          radius=x, stickiness=0.15) for x in radius]

# see an even BETTER APPROACH at the end using pandas.DataFrame

In [ ]:
# prepare the sensor and model

model = make_model("iba", "dort")
sensor = sensor_list.passive(37e9, 55)

#run!

Now we have a list of snowpacks, we want to call the model for each snowpack. We can use list comprehension again.

In [ ]:
# a NAIVE APPROACH
results = [model.run(sensor, sp) for sp in snowpack]

This return a list of results. To extract the TB V for each result can be done with another list comprehension. And then we plot the results.

In [ ]:
# still the NAIVE APPROACH
tbv = [res.TbV() for res in results]
plt.figure()
plt.plot(radius, tbv)

Nice ? We can do much better because `Model` can directly run on a list of snowpacks. It does not return a list of results, but **a unique result with a new coordinate** which is much more convenient.

In [ ]:
# a BETTER APPROACH

results = model.run(sensor, snowpack, snowpack_dimension=('radius', radius))
print(type(results))  # look results is a Result, not a list
print(results.coords) # look, we have several coordinates, one is call corr_legn

This is more compact and nicer, `results` explicitly show the radius dimension. Plotting is thus easier:

In [ ]:
plt.figure()
plt.plot(results.radius, results.TbV())

And it is easy to save all the result to disk:

In [ ]:
results.save("radius-sensitivity.nc")

Using pandas.DataFrame
-----------------------

In [ ]:
# here we build a simple DataFrame with the radius. More complex sensitivity analysis with more variables is possible
# for instance radius and density could co-vary.

sp = pd.DataFrame({'radius' : np.arange(0.05, 0.5, 0.01) * 1e-3})

sp['snowpack'] = [make_snowpack([1000.0], "sticky_hard_spheres", 
                          density=density, temperature=265,
                          radius=row['radius'], stickiness=0.15) for i, row in sp.iterrows()]

In [ ]:
results = model.run(sensor, sp).to_dataframe()

# that's it
results

It is recommended to use a named sensor with a channel_map (e.g. amsre, smos, ...) as defined in smrt.sensor.list. In this case the columns of the DataFrame are the channels of the sensor. It is a very convenient way to run multiple simulations and use the results for plotting or stats.

Recap:
---------

In [ ]:
# with List
snowpack = [make_snowpack([1000.0], "sticky_hard_spheres", density=density, temperature=265, radius=x, stickiness=0.15) for x in radius]

model = make_model("iba", "dort")
sensor = sensor_list.passive([19e9, 37e9], 55)

results = model.run(sensor, snowpack, snowpack_dimension=('radius', radius))

plt.figure()
plt.plot(results.radius, results.TbV(frequency=19e9), label="19 GHz")
plt.plot(results.radius, results.TbV(frequency=37e9), label="37 GHz")
plt.legend()

In [ ]:
# with DataFrame
sp = pd.DataFrame({'radius' : np.arange(0.05, 0.5, 0.01) * 1e-3})

sp['snowpack'] = [make_snowpack([1000.0], "sticky_hard_spheres", 
                          density=density, temperature=265,
                          radius=row['radius'], stickiness=0.15) for i, row in sp.iterrows()]

model = make_model("iba", "dort")
sensor = sensor_list.amsre(['19', '37'])

results = model.run(sensor, sp).to_dataframe()

plt.figure()
plt.plot(results['radius'], results['19V'], label="19 GHz")
plt.plot(results['radius'], results['37V'], label="37 GHz")
plt.legend()